# Stock Trading Bot Ayyyyy
- api reference: https://www.alphavantage.co/documentation/

In [ ]:
#This is just importing certain libraries we will use consistently throughouit the porject
import pandas as pd
import numpy as np
import json
import random
import requests

#### In order to access the data from the API, we need the following:
- api key: It's generated by the platform as a hash code that is unique to each user so it knows that they are authenticated to use the service
- Base URL: For all calls to the api, we use the same base url(`https://www.alphavantage.co/query?`). However, in order to get more specified data, such as weekly data or data from a certain stock ticker, we need to add that to the url as parameters. Since all will use the `base_url`, we instantiate a global variable that be used wherever we need in any function and so we don't have to keep re-pasting it.
- tickers: this is just an array of the s&p 500 tickers. We will obviously need an array of them because we want to examine multiple stocks, not just one

In [35]:
api_key = "FI0W1W7TPDCRLO18"
base_url = "https://www.alphavantage.co/query?"
tickers = ['IBM', 'AAPL'] # this is just a test for now. Will add many more later, possibly thru another api

In [36]:
def weekly_req(ticker_arr):
    """This function takes in 1 parameter, the ticker array, and makes a request to the api with every single stock ticker symbol
    that is inside of the tickers array we creatred in the above cell."""
    function = "TIME_SERIES_DAILY_ADJUSTED"
    dfs = []
    for ticker in tickers:
        url = f"{base_url}function={function}&symbol={ticker}&apikey={api_key}"
        response = requests.get(url)
        data = response.json()
        # Extract the desired data from the json response and convert it to a DataFrame
        time_series_data = data["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series_data, orient='index')
        df['symbol'] = ticker  # Add a 'symbol' column with the ticker symbol

        # Append the DataFrame to the list
        dfs.append(df)
    result_df = pd.concat(dfs)

    return result_df

In [43]:
df=weekly_req(tickers)
df.reset_index(inplace=True)

In [44]:
df.head()

,index,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,symbol
0,2023-06-16,139.23,139.469,137.47,137.48,137.48,7473676,0.0000,1.0,IBM
1,2023-06-15,137.27,138.8,137.175,138.4,138.4,3812582,0.0000,1.0,IBM
2,2023-06-14,137.8,138.93,136.94,137.2,137.2,4514888,0.0000,1.0,IBM
3,2023-06-13,136.51,138.17,136.0,137.6,137.6,3927331,0.0000,1.0,IBM
4,2023-06-12,136.0,136.62,135.8216,136.42,136.42,4500120,0.0000,1.0,IBM
